In [ ]:
import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

from gensim.models import Word2Vec

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

import seaborn as sns
import matplotlib.pyplot as plt

import os
import pandas as pd
from tqdm import *
from scipy import stats

from pathlib import Path

NathanPath="d:\Documents\Info\INF554\INF554_Kaggle_Project"
NathanPath="/users/eleves-a/2019/nathan.peluso/INF554/INF554_Kaggle_Project"

In [ ]:
project_path = str(Path(os.getcwd()).parent.absolute())
print("Current directory : " + os.getcwd() + ", Project directory : " + project_path)

In [ ]:
os.chdir(project_path)
os.chdir(NathanPath)

In [ ]:

# Read training data
df_train = pd.read_csv('data/train.csv', dtype={'author': np.int64, 'hindex': np.float32})
n_train = df_train.shape[0]

# Read test data
df_test = pd.read_csv('data/test.csv', dtype={'author': np.int64})
n_test = df_test.shape[0]

In [ ]:
G = nx.read_edgelist('data/coauthorship.edgelist', delimiter=' ', nodetype=int)
n=G.number_of_nodes()
print("Number of nodes : " + str(n))

abs_nodeID_Graph=dict(enumerate(G.nodes))
nodeID_abs_Graph=dict([(b,a) for a,b in enumerate(G.nodes)])

In [152]:
abs_nodeID_Train=dict(df_train["author"])
abs_hindex_Train=dict(df_train["hindex"])
nodeID_abs_Train=dict([(b,a) for a,b in abs_nodeID_Train.items()])

abs_nodeID_Test=dict(df_test["author"])
nodeID_abs_Test=dict([(b,a) for a,b in abs_nodeID_Test.items()])

In [ ]:
def random_walk(G, node, walk_length):
    # Simulates a random walk of length "walk_length" starting from node "node"
    walk=[node]
    for _ in range(walk_length):
        node=np.random.choice(list(G.neighbors(node)))
        walk.append(node)
    return walk

In [ ]:
def generate_walks(G, num_walks, walk_length):
    # Runs "num_walks" random walks from each node
    walks = []
    for x in G.nodes():
        for _ in range(num_walks):
            walks.append(random_walk(G,x,walk_length))
    np.random.shuffle(walks)
    return walks

In [ ]:
def deepwalk(G, num_walks, walk_length, n_dim):
    # Simulates walks and uses the Skipgram model to learn node representations
    print("Generating walks")
    walks = generate_walks(G, num_walks, walk_length)
    print("Training word2vec")
    model = Word2Vec(vector_size=n_dim, window=8, min_count=0, sg=1, workers=8, hs=1)
    model.build_vocab(walks)
    model.train(walks, total_examples=model.corpus_count, epochs=5)
    return model

n_dim = 128
n_walks = 10
walk_length = 10
#model = deepwalk(G, n_walks, walk_length, n_dim) 
#model.save("DeepWalk/Models/model_"+str(n_walks)+'_'+str(walk_length)+'_'+str(n_dim))

In [ ]:
#model=Word2Vec.load("DeepWalk/Models/model_"+str(n_walks)+'_'+str(walk_length)+'_'+str(n_dim))
model=Word2Vec.load("DeepWalk/Models/model_10_10_128")

In [ ]:
print(len(model.wv), G.number_of_nodes())
list(model.wv.vectors)

In [59]:
n=G.number_of_nodes()
DeepWalk_embeddings = np.empty(shape=(n, n_dim))

print("Filling embeddings")
for node in nodeID_abs_Graph.keys():
    DeepWalk_embeddings[nodeID_abs_Graph[node]]=model.wv.get_vector(node)


Filling embeddings


In [134]:
def MSE(X,Y):
    if (len(X)!=len(Y)):
        print("Sizes not identical")
        return -1
    return (X-Y)@(X-Y) / len(X)

In [62]:
n=abs_nodeID_Train.__len__()

idx=np.random.permutation(n)
#Careful, those indexes are related to the TRAIN set, not to the global graph indexing
idx_train=idx[:int(0.8*n)]
idx_val=idx[int(0.8*n):]

nodes_train=[abs_nodeID_Train[i] for i in idx_train]
nodes_val=[abs_nodeID_Train[i] for i in idx_val]

X_train_x = [DeepWalk_embeddings[nodeID_abs_Graph[i]] for i in nodes_train]
X_val_x = [DeepWalk_embeddings[nodeID_abs_Graph[i]] for i in nodes_val]

hindex_train_x=[abs_hindex_Train[i] for i in idx_train]
hindex_val_x=[abs_hindex_Train[i] for i in idx_val]

lin_reg=LinearRegression()
lin_reg.fit(X_train_x,hindex_train_x)
_pred=lin_reg.predict(X_val_x)
score=MSE(hindex_val_x,_pred)


print("Accuracy using DeepWalk embeddings ", score)

Accuracy using DeepWalk embeddings  138.3017125750667


## Generating submission

In [111]:
X_train_glob = [DeepWalk_embeddings[nodeID_abs_Graph[node]] for node in nodeID_abs_Train.keys()]
hindex_train_glob=[abs_hindex_Train[nodeID_abs_Train[node]] for node in nodeID_abs_Train.keys()]
X_test=[DeepWalk_embeddings[nodeID_abs_Graph[node]] for node in nodeID_abs_Test.keys()]
nodes_train=[node for node in nodeID_abs_Test.keys()]

In [84]:
lin_reg=LinearRegression()
lin_reg.fit(X_train_glob,hindex_train_glob)
_pred=lin_reg.predict(X_test)

In [125]:
len(_pred)

43560

In [112]:
submission=dict([(nodes_train[i], _pred[i]) for i in range(len(X_val_x))])

In [113]:
with open("submissions/deepwalk_lin_submission.csv", 'w') as f:
    f.write("author,hindex\n")
    for k,h in submission.items():
        f.write(str(k)+","+str(h)+"\n")
    f.close()

## Comparing

In [127]:
df_baseline = pd.read_csv('submissions/baseline_submission.csv', dtype={'author': np.float64})
baseline_dict=dict(np.array(df_baseline))
df_mine = pd.read_csv('submissions/deepwalk_lin_submission.csv', dtype={'author': np.float64})
mine_dict=dict(np.array(df_mine))

In [123]:
len(baseline_dict.keys())

43560

In [131]:
base=[baseline_dict.get(node) for node in baseline_dict.keys()]
mine_reordered=[mine_dict.get(node) for node in baseline_dict.keys()]

In [135]:
MSE(base,mine_reordered)

TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [142]:
s=0
for node in baseline_dict.keys():
    if mine_dict.get(int(node))==None:
        print(node)
        s+=1

print(s)

2347155857.0
2027724106.0
1977673974.0
2222813310.0
2103285714.0
2563858372.0
2032591291.0
2716175634.0
2109723220.0
2177680469.0
2230063939.0
1893771647.0
225151454.0
2223808495.0
1977607796.0
2225851477.0
2132268844.0
666544920.0
282210725.0
2549201646.0
2066833720.0
1997686911.0
196720240.0
2165302501.0
2181804685.0
2111399090.0
2145920534.0
2023605877.0
2557550483.0
2105493914.0
2760843174.0
2109229648.0
2117991615.0
2253343819.0
2084146335.0
2270788141.0
2159367137.0
2615570288.0
215880179.0
2461529751.0
2894241961.0
2070896388.0
2179095091.0
2083037548.0
39946086.0
46312170.0
226093892.0
2075078464.0
2022821084.0
2130479822.0
2120448159.0
2464252921.0
2858326396.0
298639177.0
2154845316.0
2153389233.0
2160474351.0
2105491491.0
2226362109.0
1994796638.0
2100987970.0
2561257150.0
1971863911.0
2110621311.0
706166500.0
2423863060.0
2265808451.0
2595503054.0
1484374571.0
2244183524.0
1978447724.0
2143673831.0
2400854912.0
2858894105.0
2122019980.0
80435470.0
2141506386.0
2029023891.0
